In [8]:
#!pip install autogluon

In [9]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold
import numpy as np

In [10]:
x_train_a = pd.read_csv('cleaned_data/A/x_train_a.csv')
y_train_a = pd.read_csv('cleaned_data/A/train_a.csv')
x_test_a = pd.read_csv('cleaned_data/A/x_test_a.csv')

In [11]:
x_train_b = pd.read_csv('cleaned_data/B/x_train_b.csv')
y_train_b = pd.read_csv('cleaned_data/B/train_b.csv')
x_test_b = pd.read_csv('cleaned_data/B/x_test_b.csv')

x_train_b

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,month_sin,month_cos,day_of_year,day_of_year_sin,day_of_year_cos,direct_rad_3h_roll_avg,diffuse_rad_3h_roll_avg,direct_rad_6h_roll_avg,diffuse_rad_6h_roll_avg,direct_rad_x_sun_elevation
0,2019-01-01 00:00:00,5.525,1.23975,1200.6750,0.0,0.000,1200.6750,0.0,275.150,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
1,2019-01-01 01:00:00,5.425,1.23975,1131.4249,0.0,0.000,1131.4249,0.0,274.825,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
2,2019-01-01 02:00:00,5.400,1.23850,1061.0000,0.0,0.000,1061.0000,0.0,274.800,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
3,2019-01-01 03:00:00,5.350,1.23975,1021.1500,0.0,0.000,1021.1500,0.0,274.675,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
4,2019-01-01 04:00:00,5.675,1.23750,1033.7000,0.0,0.000,1033.7000,0.0,275.500,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26956,2023-04-30 19:00:00,4.550,1.27650,1677.9500,337850.1,4.225,542.8500,0.0,272.425,2.825,...,0.866025,-0.500000,120,0.882679,-0.469977,20.058333,35.033333,83.608334,94.829166,-0.01805
26957,2023-04-30 20:00:00,4.500,1.27875,1766.5000,9083.1,0.000,546.3500,0.0,272.300,0.000,...,0.866025,-0.500000,120,0.882679,-0.469977,2.458333,10.458333,52.808334,63.945835,-0.00000
26958,2023-04-30 21:00:00,4.500,1.27900,1698.9250,0.0,0.000,548.0500,0.0,272.300,0.000,...,0.866025,-0.500000,120,0.882679,-0.469977,0.033333,0.941667,26.433334,37.866667,-0.00000
26959,2023-04-30 22:00:00,4.500,1.27975,1354.8250,0.0,0.000,527.6000,0.0,272.275,0.000,...,0.866025,-0.500000,120,0.882679,-0.469977,0.000000,0.000000,10.029167,17.516666,-0.00000


In [12]:
x_train_c = pd.read_csv('cleaned_data/C/x_train_c.csv')
y_train_c = pd.read_csv('cleaned_data/C/train_c.csv')
x_test_c = pd.read_csv('cleaned_data/C/x_test_c.csv')

In [13]:
class CustomTimeSeriesSplit(_BaseKFold):
    def __init__(self, n_splits, train_size=None, test_size=None):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.train_size = train_size
        self.test_size = test_size

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        indices = np.arange(n_samples)

        # Define initial sizes if not provided
        train_size = self.train_size or n_samples // (self.n_splits + 1)
        test_size = self.test_size or n_samples // self.n_splits

        for test_start in range(train_size + test_size, n_samples, test_size):
            train_end = test_start - test_size
            train_start = max(train_end - train_size, 0)
            yield indices[train_start:train_end], indices[train_end:test_start]

# Example usage
# tscv = CustomTimeSeriesSplit(n_splits=5, train_size=720, test_size=120)

# for train_idx, test_idx in tscv.split(x_train_a):
#     print("Train indices:", train_idx)
#     print("Test indices:", test_idx)

In [14]:
x_train_a_combined = x_train_a.merge(y_train_a, left_on='date_forecast', right_on='time', how='left')
x_train_a_combined['observed'] = x_train_a_combined['calc_year'].isna().astype(int)
train_data_a = x_train_a_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'direct_rad_x_sun_elevation','day_of_year', 'forecast_year','forecast_month','forecast_day','forecast_hour'], axis = 1)
x_test_a['observed'] = x_test_a['calc_year'].isna().astype(int)
test_data_a = x_test_a.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'direct_rad_x_sun_elevation','day_of_year','forecast_year','forecast_month','forecast_day','forecast_hour'], axis = 1)

In [15]:
x_train_b_combined = x_train_b.merge(y_train_b, left_on='date_forecast', right_on='time', how='left')
x_train_b_combined['observed'] = x_train_b_combined['calc_year'].isna().astype(int)

train_data_b = x_train_b_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'direct_rad_x_sun_elevation','day_of_year','forecast_year','forecast_month','forecast_day','forecast_hour'], axis = 1)
x_test_b['observed'] = x_test_b['calc_year'].isna().astype(int)
test_data_b = x_test_b.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'direct_rad_x_sun_elevation','day_of_year','forecast_year','forecast_month','forecast_day','forecast_hour'], axis = 1)

In [16]:
x_train_c_combined = x_train_c.merge(y_train_c, left_on='date_forecast', right_on='time', how='left')
x_train_c_combined['observed'] = x_train_c_combined['calc_year'].isna().astype(int)
train_data_c = x_train_c_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'direct_rad_x_sun_elevation','day_of_year','forecast_year','forecast_month','forecast_day','forecast_hour'], axis = 1)
x_test_c['observed'] = x_test_c['calc_year'].isna().astype(int)
test_data_c = x_test_c.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'direct_rad_x_sun_elevation','day_of_year','forecast_year','forecast_month','forecast_day','forecast_hour'], axis = 1)

In [17]:
test_data_b

,location,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,observed
0,B,4.300,1.28300,912.3000,0.0,0.000,1059.750,0.0,271.65002,0.000,...,31107.600,3.950,2.100,3.375,0.0,2023,5,1,0,0
1,B,4.250,1.28300,1482.8002,0.0,0.000,1073.700,0.0,271.45000,0.000,...,30409.700,3.825,1.925,3.300,0.0,2023,5,1,1,0
2,B,4.150,1.28275,1765.9000,0.0,0.000,1200.100,0.0,271.05000,0.000,...,31342.650,3.650,1.750,3.225,0.0,2023,5,1,2,0
3,B,4.025,1.28225,2269.7500,40510.2,11.675,1179.000,0.0,270.65000,9.375,...,34475.523,3.500,1.475,3.150,0.0,2023,5,1,3,0
4,B,3.900,1.28200,2198.2250,567057.1,76.900,919.150,0.0,270.37500,47.400,...,35069.074,3.325,1.300,3.075,0.0,2023,5,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,B,8.350,1.19800,3640.1250,1908360.9,85.100,2015.750,0.0,281.57500,33.625,...,43617.250,2.475,2.075,-1.350,0.0,2023,7,3,19,0
716,B,8.525,1.20075,3351.1000,737351.8,24.800,1613.375,0.0,281.85000,14.350,...,43830.600,2.450,2.100,-1.275,0.0,2023,7,3,20,0
717,B,8.800,1.20375,2753.0250,149728.8,1.275,1624.450,0.0,282.30000,1.300,...,43401.027,2.575,2.150,-1.400,0.0,2023,7,3,21,0
718,B,9.000,1.20600,2204.5000,1440.5,0.000,1768.325,0.0,282.67502,0.000,...,40885.900,2.250,1.800,-1.350,0.0,2023,7,3,22,0


In [25]:
train_data_a

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed
0,2019-06-02 22:00:00,7.700,1.22825,1728.950,0.0,0.000,1728.950,0.0,280.30000,0.000,...,3.600,-3.575,-0.500,0.0,2019,6,2,22,0.00,1
1,2019-06-02 23:00:00,7.700,1.22350,1689.825,0.0,0.000,1689.825,0.0,280.30000,0.000,...,3.350,-3.350,0.275,0.0,2019,6,2,23,0.00,1
2,2019-06-03 00:00:00,7.875,1.21975,1563.225,0.0,0.000,1563.225,0.0,280.65000,0.000,...,3.050,-2.950,0.750,0.0,2019,6,3,0,0.00,1
3,2019-06-03 01:00:00,8.425,1.21800,1283.425,834.6,0.750,1283.425,0.0,281.67500,0.300,...,2.725,-2.600,0.875,0.0,2019,6,3,1,0.00,1
4,2019-06-03 02:00:00,8.950,1.21800,1003.500,129872.6,23.100,1003.500,0.0,282.50000,11.975,...,2.550,-2.350,0.925,0.0,2019,6,3,2,19.36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31883,2023-01-29 06:00:00,5.125,1.26000,1747.100,0.0,0.000,1091.250,0.0,274.05000,0.000,...,3.600,1.275,3.350,0.0,2023,1,29,6,0.00,0
31884,2023-01-29 07:00:00,5.200,1.25800,1342.025,0.0,0.000,1084.025,0.0,274.20000,0.000,...,3.350,0.800,3.250,0.0,2023,1,29,7,0.00,0
31885,2023-01-29 08:00:00,5.175,1.25575,1255.175,19398.6,6.625,974.000,0.0,274.15002,1.625,...,3.050,0.300,3.050,0.0,2023,1,29,8,0.88,0
31886,2023-01-29 09:00:00,5.175,1.25300,1177.000,320519.0,40.525,916.350,0.0,274.15000,8.175,...,2.675,0.250,2.650,0.0,2023,1,29,9,52.80,0


# Trying with validation set equal to half of the estimated data


In [18]:
def split_dataset(train_data, observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set contains the last half of the rows where observed = 0,
    and the training set contains the rest.

    :param train_data: The original training dataset as a pandas DataFrame.
    :param observed_column: The name of the column that indicates if the row is observed.
    :return: A tuple (training_set, validation_set)
    """

    # Filter rows where observed = 0
    observed_zero = train_data[train_data[observed_column] == 0]

    # Split the filtered dataset into two
    half_index = len(observed_zero) // 2
    validation_set = observed_zero[half_index:]

    # Combine the first half of observed_zero with the rest of the data where observed != 0
    training_set = pd.concat([train_data[train_data[observed_column] != 0], observed_zero[:half_index]])

    return training_set, validation_set


# Example usage:
# train_data_a, val_data_a = split_dataset(train_data_a, 'observed')
# train_data_b, val_data_b = split_dataset(train_data_b, 'observed')
# train_data_c, val_data_c = split_dataset(train_data_c, 'observed')


In [31]:

#val_data_a.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed
31888,2023-01-29 11:00:00,5.375,1.24575,1226.200,1258656.200,93.950,804.000,0.0,274.700,16.325,...,2.200,0.350,2.15,0.0,2023,1,29,11,68.20,0
31889,2023-01-29 12:00:00,5.500,1.24150,1717.325,1315152.800,84.200,946.400,0.0,275.050,11.450,...,2.450,-0.100,2.45,0.0,2023,1,29,12,37.18,0
31890,2023-01-29 13:00:00,5.650,1.23650,1940.650,997982.700,51.750,922.575,0.0,275.425,8.675,...,2.475,-0.475,2.45,0.0,2023,1,29,13,33.66,0
31891,2023-01-29 14:00:00,5.750,1.23175,1281.750,470574.600,14.425,957.025,0.0,275.700,2.200,...,3.100,0.000,3.05,0.0,2023,1,29,14,1.54,0
31892,2023-01-29 15:00:00,5.850,1.22725,1558.850,63008.402,0.000,1202.300,0.0,275.975,0.000,...,3.725,0.950,3.60,0.0,2023,1,29,15,0.00,0


In [20]:
predictor_a= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_a,verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231102_153411\"
Presets specified: ['best_quality']
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231102_153411\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   706.87 GB / 1022.87 GB (69.1%)
Train Data Rows:    31888
Train Data Columns: 51
Tuning Data Rows:    2197
Tuning Data Columns: 51
Label Column: pv_measurement
Preprocessing data ...
c:\Users\holwe\mini

KeyboardInterrupt: 

In [ ]:
predictor_b= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_b, verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231101_115216\"
Presets specified: ['best_quality']
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231101_115216\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   705.97 GB / 1022.87 GB (69.0%)
Train Data Rows:    25160
Train Data Columns: 51
Tuning Data Rows:    1801
Tuning Data Columns: 51
Label Column: pv_measurement
Preprocessing data ...
c:\Users\holwe\mini

In [ ]:
predictor_c= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_c, verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231101_121349\"
Presets specified: ['best_quality']
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231101_121349\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   702.12 GB / 1022.87 GB (68.6%)
Train Data Rows:    20219
Train Data Columns: 51
Tuning Data Rows:    1465
Tuning Data Columns: 51
Label Column: pv_measurement
Preprocessing data ...
c:\Users\holwe\mini

In [ ]:
predictor_a.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-178.082495,174.447727,1127.122275,0.000000,0.329203,3,True,17
1,NeuralNetTorch_BAG_L2,-178.523082,174.126863,1120.951374,0.628202,128.027010,2,True,15
2,LightGBMXT_BAG_L2,-183.667943,173.819525,998.766062,0.320865,5.841698,2,True,8
3,WeightedEnsemble_L2,-184.475402,172.704435,993.102960,0.001001,0.238128,2,True,7
4,CatBoost_BAG_L2,-186.801453,173.552667,1008.368154,0.054006,15.443790,2,True,11
5,LightGBM_BAG_L1,-186.857859,80.196621,70.830802,80.196621,70.830802,1,True,4
6,XGBoost_BAG_L2,-187.288225,173.666728,998.581906,0.168068,5.657542,2,True,14
7,NeuralNetFastAI_BAG_L2,-188.404671,174.267970,1054.560816,0.769310,61.636452,2,True,13
8,LightGBM_BAG_L2,-189.213515,173.616261,997.128823,0.117600,4.204460,2,True,9
9,LightGBMLarge_BAG_L2,-191.091235,173.894806,1006.840367,0.396145,13.916003,2,True,16


In [ ]:
predictor_b.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-27.228804,68.250001,122.967029,0.000000,0.309280,2,True,11
1,WeightedEnsemble_L3,-27.553844,199.093395,1046.030706,0.000000,0.259099,3,True,21
2,LightGBMXT_BAG_L2,-27.672920,197.249926,941.266932,0.293107,5.819569,2,True,12
3,XGBoost_BAG_L2,-28.034938,197.124955,943.987183,0.168136,8.539820,2,True,18
4,LightGBM_BAG_L2,-28.050321,197.059256,939.219580,0.102437,3.772217,2,True,13
5,NeuralNetFastAI_BAG_L2,-28.074649,197.731210,985.959046,0.774390,50.511683,2,True,17
6,CatBoost_BAG_L2,-28.090788,197.010667,953.899405,0.053848,18.452042,2,True,15
7,RandomForestMSE_BAG_L2,-28.093414,198.025898,989.440355,1.069079,53.992992,2,True,14
8,LightGBMXT_BAG_L1,-28.161444,66.580985,67.869242,66.580985,67.869242,1,True,3
9,ExtraTreesMSE_BAG_L2,-28.231372,198.047545,944.162179,1.090725,8.714816,2,True,16


In [ ]:
predictor_c.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-33.472780,109.628170,324.913947,0.001000,0.307574,2,True,11
1,NeuralNetTorch_BAG_L1,-35.054357,0.452867,74.048636,0.452867,74.048636,1,True,10
2,WeightedEnsemble_L3,-36.204462,155.216634,991.370745,0.000000,0.259040,3,True,21
3,NeuralNetTorch_BAG_L2,-36.204462,155.216634,991.111705,0.410168,59.868227,2,True,19
4,NeuralNetFastAI_BAG_L2,-37.779710,155.406234,978.494639,0.599767,47.251161,2,True,17
5,CatBoost_BAG_L2,-37.958021,154.849001,951.434585,0.042534,20.191107,2,True,15
6,XGBoost_BAG_L2,-38.565512,154.930502,938.236235,0.124036,6.992757,2,True,18
7,LightGBM_BAG_L2,-39.309482,154.907189,935.822145,0.100723,4.578667,2,True,13
8,XGBoost_BAG_L1,-39.694971,42.065310,187.682290,42.065310,187.682290,1,True,9
9,ExtraTreesMSE_BAG_L2,-39.721547,155.906317,938.652058,1.099851,7.408581,2,True,16


In [ ]:
nan_counts = x_train_a_combined.isna().sum()
print(nan_counts)

date_forecast                    0
absolute_humidity_2m:gm3        24
air_density_2m:kgm3             24
ceiling_height_agl:m          6151
clear_sky_energy_1h:J            0
                              ... 
diffuse_rad_6h_roll_avg         19
direct_rad_x_sun_elevation      24
time                             0
pv_measurement                   0
observed                         0
Length: 69, dtype: int64


In [ ]:
nan_counts = x_train_b_combined.isna().sum()
print(nan_counts)

date_forecast                    0
absolute_humidity_2m:gm3        24
air_density_2m:kgm3             24
ceiling_height_agl:m          4229
clear_sky_energy_1h:J            0
                              ... 
diffuse_rad_6h_roll_avg         19
direct_rad_x_sun_elevation      24
time                             0
pv_measurement                   0
observed                         0
Length: 69, dtype: int64


In [ ]:
nan_counts = x_train_c_combined.isna().sum()
print(nan_counts)

date_forecast                    0
absolute_humidity_2m:gm3        24
air_density_2m:kgm3             24
ceiling_height_agl:m          4457
clear_sky_energy_1h:J            0
                              ... 
diffuse_rad_6h_roll_avg         19
direct_rad_x_sun_elevation      24
time                             0
pv_measurement                   0
observed                         0
Length: 69, dtype: int64


In [ ]:
#predictor_a.feature_importance(train_)

In [ ]:
def aggregate_rows(df, value_column='pv_measurement'):
    # Determine the number of rows in the dataframe
    n_rows = len(df)

    # Ensure that the number of rows is a multiple of 4
    if n_rows % 4 != 0:
        print(f"Warning: Number of rows ({n_rows}) is not a multiple of 4. Truncating to {n_rows // 4 * 4}.")
        df = df.iloc[:n_rows // 4 * 4]

    # Group by each set of four rows and sum the values
    grouped_values = df[value_column].groupby(df.index // 4).sum()

    # Create a new DataFrame with the summed values
    df_aggregated = pd.DataFrame(grouped_values).reset_index(drop=True)

    return df_aggregated



In [ ]:
y_pred_a = predictor_a.predict(test_data_a)
y_pred_a=pd.DataFrame(y_pred_a,columns=['pv_measurement']) 
y_pred_a #print the DataFrame

,pv_measurement
0,0.181013
1,0.160474
2,0.147849
3,68.233101
4,454.940552
...,...
715,217.218506
716,79.877754
717,10.130502
718,0.340277


In [ ]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a f

,pv_measurement
0,-0.495047
1,-0.596373
2,-0.682201
3,5.277022
4,48.522602
...,...
715,41.507229
716,14.737422
717,2.852947
718,0.810264


In [ ]:
y_pred_c = predictor_c.predict(test_data_c)
y_pred_c=pd.DataFrame(y_pred_c,columns=['pv_measurement'])
y_pred_c #print the DataFrame

,pv_measurement
0,0.023266
1,0.014784
2,0.034705
3,1.703338
4,26.780682
...,...
715,23.893417
716,9.925955
717,2.743544
718,0.170082


In [ ]:
predictions = pd.concat([y_pred_a,y_pred_b, y_pred_c], ignore_index= True)
predictions.tail()

,pv_measurement
2155,23.893417
2156,9.925955
2157,2.743544
2158,0.170082
2159,0.164730


In [ ]:
predictions_df = predictions.rename(columns={'pv_measurement': 'prediction'})

sample_submission = pd.read_csv('sample_submission.csv')

# Convert all negative predictions to 0
predictions_df.loc[predictions_df['prediction'] < 0, 'prediction'] = 0

# Join the 'id' column from sample_submission with the predictions
sample_submission['prediction'] = predictions_df['prediction']

# Save to CSV
sample_submission.to_csv('autogluon_prediction.csv', index=False)
predictions_df

,prediction
0,0.181013
1,0.160474
2,0.147849
3,68.233101
4,454.940552
...,...
2155,23.893417
2156,9.925955
2157,2.743544
2158,0.170082


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

performance_metrics = []


for train_idx, test_idx in tscv.split(x_train_a):
    x_train_fold = x_train_a.iloc[train_idx]
    y_train_fold = y_train_a.iloc[train_idx]
    x_valid_fold = x_train_a.iloc[test_idx]
    y_valid_fold = y_train_a.iloc[test_idx]

    # Merge the data on the date
    train_data_combined = x_train_fold.merge(y_train_fold, left_on='date_forecast', right_on='time', how='left')
    valid_data_combined = x_valid_fold.merge(y_valid_fold, left_on='date_forecast', right_on='time', how='left')

    # Drop redundant columns
    train_data_combined.drop(columns=['time'], inplace=True)
    valid_data_combined.drop(columns=['time'], inplace=True)


    # Define the name of the target column
    label = 'pv_measurement'  # Replace with the name of your target column
    
    # Train the model
    predictor = TabularPredictor(label=label, eval_metric= 'mean_absolute_error').fit(train_data_combined,tuning_data= valid_data_combined, time_limit= 900)
    
    # Validate the model
    performance = predictor.evaluate(valid_data_combined)
    
    # Evaluate the performance (you can use any metric of your choice)

    performance_metrics.append(performa)



No path specified. Models will be saved in: "AutogluonModels\ag-20231101_123447\"
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels\ag-20231101_123447\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   701.94 GB / 1022.87 GB (68.6%)
Train Data Rows:    5685
Train Data Columns: 66
Label Column: pv_measurement
Preprocessing data ...
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN bef

	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\features\generators\fillna.py:58: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X.fillna(self._fillna_feature_map, inplace=True, downcast=False)
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 6): ['elevation:m', 'snow_drift:idx', 'calc_year', 'calc_month', 'calc_day', 'calc_hour']
		These features carry no predictive signal and should be m

[1000]	valid_set's l1: 149.482
[2000]	valid_set's l1: 144.155
[3000]	valid_set's l1: 141.946
[4000]	valid_set's l1: 141.201
[5000]	valid_set's l1: 140.785
[6000]	valid_set's l1: 140.63
[7000]	valid_set's l1: 140.503
[8000]	valid_set's l1: 140.403
[9000]	valid_set's l1: 140.364
[10000]	valid_set's l1: 140.332


	-140.3307	 = Validation score   (-mean_absolute_error)
	7.74s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 891.12s of the 891.12s of remaining time.


[1000]	valid_set's l1: 152.851
[2000]	valid_set's l1: 152.764
[3000]	valid_set's l1: 152.646
[4000]	valid_set's l1: 152.628
[5000]	valid_set's l1: 152.638


...

[1000]	valid_set's l1: 166.693
[2000]	valid_set's l1: 157.659
[3000]	valid_set's l1: 154.758
[4000]	valid_set's l1: 152.936
[5000]	valid_set's l1: 152.012
[6000]	valid_set's l1: 151.61
[7000]	valid_set's l1: 151.103
[8000]	valid_set's l1: 150.77
[9000]	valid_set's l1: 150.456
[10000]	valid_set's l1: 150.259


	-150.2422	 = Validation score   (-mean_absolute_error)
	11.24s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 887.11s of the 887.11s of remaining time.


[1000]	valid_set's l1: 169.592
[2000]	valid_set's l1: 165.632
[3000]	valid_set's l1: 164.666
[4000]	valid_set's l1: 164.109
[5000]	valid_set's l1: 163.818
[6000]	valid_set's l1: 163.624
[7000]	valid_set's l1: 163.502
[8000]	valid_set's l1: 163.467
[9000]	valid_set's l1: 163.414
[10000]	valid_set's l1: 163.415


	-163.4101	 = Validation score   (-mean_absolute_error)
	13.89s	 = Training   runtime
	0.27s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 872.52s of the 872.52s of remaining time.
	-192.4784	 = Validation score   (-mean_absolute_error)
	36.77s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 835.42s of the 835.42s of remaining time.


KeyboardInterrupt: 

No path specified. Models will be saved in: "AutogluonModels\ag-20231026_153540\"


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│   1 # Begin with A                                                                               │
│   2                                                                                              │
│ ❱ 3 predictor = TabularPredictor(label='pv_measurement').fit(train_data=x_train, tuning_data     │
│   4 predictions = predictor.predict()  # Assuming x_test is your test data                       │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'x_train' is not defined